In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate synthetic data with 3 classes
n_samples = 1000
n_classes = 3
X, y = make_blobs(n_samples=n_samples, 
                  centers=n_classes, 
                  n_features=2,
                  random_state=42)

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X = torch.FloatTensor(X)
y = torch.LongTensor(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the multi-class logistic regression model
class MultiClassLogisticRegression(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MultiClassLogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, num_classes)
        
    def forward(self, x):
        return self.linear(x)

# Initialize model, loss function, and optimizer
model = MultiClassLogisticRegression(input_dim=2, num_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        with torch.no_grad():
            _, predicted = torch.max(outputs, 1)
            train_accuracy = (predicted == y_train).float().mean()
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {train_accuracy:.4f}')

# Evaluation
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).float().mean()
    print(f'Test Accuracy: {accuracy.item():.4f}')

# Visualization
plt.figure(figsize=(10, 6))
colors = ['red', 'blue', 'green']

# Plot test points
for i in range(n_classes):
    mask = y_test == i
    plt.scatter(X_test[mask, 0], X_test[mask, 1], c=colors[i], label=f'Class {i}')

# Create decision boundary
x1_min, x1_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
x2_min, x2_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.1),
                       np.arange(x2_min, x2_max, 0.1))
grid = torch.FloatTensor(np.c_[xx1.ravel(), xx2.ravel()])

with torch.no_grad():
    outputs = model(grid)
    _, predicted = torch.max(outputs, 1)
    
Z = predicted.reshape(xx1.shape)
plt.contourf(xx1, xx2, Z, alpha=0.3, levels=np.arange(n_classes + 1) - 0.5)

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Multi-Class Logistic Regression Decision Boundaries')
plt.legend()
plt.show()